In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.0.0


In [59]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)   #[60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)   #[10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)  #[60000]
        self.test_label = self.test_label.astype(np.int32)   #[10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], size=batch_size)
        return self.train_data[index, :], self.train_label[index]
    
    def get_batch_valid(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.test_data)[0], size=batch_size)
        return self.test_data[index, :], self.test_label[index]

In [79]:
class CNN(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv11 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[1, 1],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.conv12 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[3, 3],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.conv13 = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=[5, 5],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.concatenate = tf.keras.layers.Concatenate()
    self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.conv2 = tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=[3, 3],
        strides=(1, 1),
        padding='same',
        activation=tf.nn.relu
    )
    self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
    self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
    self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, inputs):
    x1 = self.conv11(inputs)  # [28, 28, 32]
    x2 = self.conv12(inputs)  # [28, 28, 32]
    x3 = self.conv13(inputs)  # [28, 28, 32]
    x = self.concatenate([x1, x2, x3]) # [28, 28, 96]
    x = self.pool1(x)    # [14, 14, 96]
    x = self.conv2(x)    # [14, 14, 64]
    x = self.pool2(x)    # [7, 7, 64]
    x = self.flatten(x)   # [7*7*64]
    x = self.dense1(x)    # [256]
    x = self.dense2(x)    # [10]
    output = tf.nn.softmax(x)
    return output

num_epochs = 5
batch_size = 32
learning_rate = 0.001

def train():
    model = CNN()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    data_loader = MNISTLoader()
    train_dataset = tf.data.Dataset.from_tensor_slices((data_loader.train_data, data_loader.train_label))
    train_dataset = train_dataset.shuffle(buffer_size=23000)
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    log_dir = "./tensorboard"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
histogram_freq=1)
    
    history = model.fit(
         train_dataset,
         epochs=num_epochs,
         verbose=1,
         callbacks=[tensorboard_callback]
         )

def train2():
    model = CNN()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                    loss=tf.keras.losses.sparse_categorical_crossentropy,
                    metrics=[tf.keras.metrics.sparse_categorical_accuracy])
    
    data_loader = MNISTLoader()
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,  # 数据集去中心化
        rotation_range=20,        # 旋转角度范围
        width_shift_range=0.2,    # 水平位置平移
        height_shift_range=0.2,   # 上下位置平移
        horizontal_flip=True)    # 随机水平翻转
    datagen.fit(data_loader.train_data)
    
    log_dir = "./tensorboard"
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
histogram_freq=1)
    
    history = model.fit_generator(
         datagen.flow(data_loader.train_data, data_loader.train_label, batch_size=batch_size),
         steps_per_epoch=len(data_loader.train_data) / batch_size,
         epochs=num_epochs,
         verbose=1,
         callbacks=[tensorboard_callback]
         )

In [80]:
if __name__ ==  '__main__':
    train2()

Epoch 1/5


ImportError: Image transformations require SciPy. Install SciPy.